In [2]:
import polars as pd
import numpy as np
from pathlib import Path

In [3]:
# Define data paths
data_dir = Path('../data/raw')
lending_club_dir = data_dir / 'lending_club'

# List all CSV files in the lending-club directory
csv_files = list(lending_club_dir.glob('*.csv'))
print("Found CSV files:")
for file in csv_files:
    print(f"  - {file.name}")
    
# Check file sizes
print("\nFile sizes:")
for file in csv_files:
    size_mb = file.stat().st_size / (1024 * 1024)
    print(f"  - {file.name}: {size_mb:.1f} MB")

Found CSV files:
  - accepted_2007_to_2018Q4.csv
  - rejected_2007_to_2018Q4.csv

File sizes:
  - accepted_2007_to_2018Q4.csv: 1597.5 MB
  - rejected_2007_to_2018Q4.csv: 1699.7 MB


In [4]:
# Load the accepted loans dataset
print("Loading accepted loans dataset...")
accepted_file = lending_club_dir / 'accepted_2007_to_2018Q4.csv'

# Load the full dataset (this might take a while for large files)
print("\nLoading full accepted dataset...")

# Fixes for parsing issues:
# - increase infer_schema_length so schema is inferred from more rows
# - treat the 'id' column as text to avoid integer parse errors when stray lines appear
# - ignore parsing errors so malformed lines don't crash the reader
# - add the problematic summary line to null_values so it's treated as missing if encountered
accepted_df = pd.read_csv(
	accepted_file,
	infer_schema_length=20000,
	schema_overrides={"id": pd.Utf8},
	ignore_errors=True,
	null_values=["Total amount funded in policy code 1: 6417608175"]
)

print(f"Full dataset shape: {accepted_df.shape}")

Loading accepted loans dataset...

Loading full accepted dataset...
Full dataset shape: (2260701, 151)


# Dataset Summary Analysis

This notebook provides a comprehensive summary of the Lending Club datasets found in the data/external folder.

In [ ]:
# Detailed statistical summary of accepted loans
print("=== ACCEPTED LOANS DATASET SUMMARY ===")
print(f"Dataset shape: {accepted_df.shape[0]:,} rows, {accepted_df.shape[1]} columns")
print(f"Date range: {accepted_df['issue_d'].min()} to {accepted_df['issue_d'].max()}" if 'issue_d' in accepted_df.columns else "No issue_d column found")

print("\n--- Data Types ---")
print(accepted_df.count())

print("\n--- Missing Values ---")
missing_values = accepted_df.null_count()
missing_pct = (missing_values / len(accepted_df)) * 100
missing_summary = pd.DataFrame({
    'Missing_Count': missing_values,
    'Missing_Percentage': missing_pct
}).sort(by='Missing_Percentage', descending=True)

print("\n--- Numerical Columns Summary ---")
numeric_cols = accepted_df.columns
accepted_df.dtypes
if len(numeric_cols) > 0:
    for col in numeric_cols:
        print(f"\nColumn: {col}")
        print(accepted_df.select(col, allow_object=True).describe())
        

=== ACCEPTED LOANS DATASET SUMMARY ===
Dataset shape: 2,260,701 rows, 151 columns
Date range: Apr-2008 to Sep-2018

--- Data Types ---
shape: (1, 151)
┌─────────┬───────────┬───────────┬────────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ id      ┆ member_id ┆ loan_amnt ┆ funded_amn ┆ … ┆ settlemen ┆ settlemen ┆ settlemen ┆ settlemen │
│ ---     ┆ ---       ┆ ---       ┆ t          ┆   ┆ t_date    ┆ t_amount  ┆ t_percent ┆ t_term    │
│ u32     ┆ u32       ┆ u32       ┆ ---        ┆   ┆ ---       ┆ ---       ┆ age       ┆ ---       │
│         ┆           ┆           ┆ u32        ┆   ┆ u32       ┆ u32       ┆ ---       ┆ u32       │
│         ┆           ┆           ┆            ┆   ┆           ┆           ┆ u32       ┆           │
╞═════════╪═══════════╪═══════════╪════════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 2260700 ┆ 0         ┆ 2260668   ┆ 2260668    ┆ … ┆ 34246     ┆ 34246     ┆ 34246     ┆ 34246     │
└─────────┴───────────┴───────────┴──────

In [20]:
accepted_df.collect_schema()

Schema([('id', String),
        ('member_id', String),
        ('loan_amnt', Float64),
        ('funded_amnt', Float64),
        ('funded_amnt_inv', Float64),
        ('term', String),
        ('int_rate', Float64),
        ('installment', Float64),
        ('grade', String),
        ('sub_grade', String),
        ('emp_title', String),
        ('emp_length', String),
        ('home_ownership', String),
        ('annual_inc', Float64),
        ('verification_status', String),
        ('issue_d', String),
        ('loan_status', String),
        ('pymnt_plan', String),
        ('url', String),
        ('desc', String),
        ('purpose', String),
        ('title', String),
        ('zip_code', String),
        ('addr_state', String),
        ('dti', Float64),
        ('delinq_2yrs', Float64),
        ('earliest_cr_line', String),
        ('fico_range_low', Float64),
        ('fico_range_high', Float64),
        ('inq_last_6mths', Float64),
        ('mths_since_last_delinq', Float64),
 

In [14]:
accepted_df['loan_status'].unique()

loan_status
str
"""Does not meet the credit polic…"
"""Default"""
"""Charged Off"""
"""Fully Paid"""
"""Does not meet the credit polic…"
"""In Grace Period"""
null
"""Late (16-30 days)"""
"""Late (31-120 days)"""


In [16]:
ls_df = (
    accepted_df
    .filter(pd.col('loan_status').is_in(['Charged Off', 'Fully Paid', 'Current']))
    .get_column('loan_status')
    .clone()
)

In [ ]:
vc = ls_df.value_counts().sort('count', descending=True)
total = len(ls_df)
for val, cnt in vc.iter_rows():
    print(f"{val}: {int(cnt)/total:.1%}")

Fully Paid: 48.4%
Current: 39.5%
Charged Off: 12.1%
